In [2]:
import json

with open("sample_config.json", "r") as sample_file:
    template = json.load(sample_file)
    
class RefPath:
    def __init__(self, ref_path):
        paths = ref_path.split("#/info/x-pods/")
        parts = paths[1].split("/")
        self.pod_name = parts[0]
        self.container_name = parts[2]

    @property
    def full_path(self):
        return "#/info/x-pods/" + self.pod_name + "/containers/" + self.container_name + "/port"


class Method:
    def __init__(self, path_name, method_name, ref_path, full_method):
        self.path_name = path_name
        self.method_name = method_name
        self.ref_path = ref_path
        self.full_method = full_method

    def __str__(self):
        return self.path_name + ": " + self.method_name + " (" + self.ref_path.full_path + ")"
    
class Pod:
    def __init__(self, pod_name, full_pod):
        self.pod_name = pod_name
        self.full_pod = full_pod

In [3]:
methods = []
for path_name, path in template["paths"].items():
    for method_name, method in path.items():        
        ref_path = RefPath(method["x-location"]["$ref"])        
        methods.append(Method(path_name, method_name, ref_path, method))

pods = []
for pod_name, pod in template["info"]["x-pods"].items():
    full_pod = dict()
    full_pod[pod_name] = pod
    pods.append(Pod(pod_name, full_pod))

In [8]:
import copy
templates = []
for pod in pods:
    sample_template = copy.deepcopy(template)
    pod_methods = {}
    for method in methods:
        if method.ref_path.pod_name == pod.pod_name:
            pod_methods.setdefault(method.path_name, {})
            pod_methods[method.path_name][method.method_name] = method.full_method
    if len(pod_methods):
        sample_template["info"]["x-pods"] = pod.full_pod
        sample_template["paths"] = pod_methods

        templates.append(sample_template)

In [19]:
def gen_dict_extract(key, var):
    if hasattr(var,'items'):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, dict):                
                for result in gen_dict_extract(key, v):
                    yield result
            elif isinstance(v, (list)):
                for d in v:
                    for result in gen_dict_extract(key, d):
                        yield result
                        
def get_schemas_only(references):
    schemas = []
    for reference in references:
        schema = reference.split("#/components/schemas/")        
        try:
            schemas.append(schema[1])
        except IndexError:
            pass

    return schemas


def get_schema(references):
    schemas = []
    for reference in references:
        schema = reference.split("#/components/schemas/")        
        try:
            return schema[1]
        except IndexError:
            pass    

In [20]:
schemas_methods = {}
for method in methods:    
    all_schemas = list(set(gen_dict_extract('$ref', method.full_method)))
    print(get_schema(all_schemas))


Pet
Pet
Pet
Pet
Pet
None
None
ApiResponse
None
Order
Order
None
User
User
None
None
User
User
None


In [17]:
get_schema(["#/components/schemas/Pets", "#/components/schemas/Users"])

['Pets', 'Users']